# A Brief History of Async

## Concurrency

Concurrency == simulating parallelism by switching context.

![concurrency](concurrency.png)

### Sequential execution

In [40]:
import httpx


def job(n):
    print(f"--- request {n} sent")
    httpx.get(f"https://example.com/{n}")
    print(f"--- response {n} received")

In [41]:
%%time

job(1)
job(2)

--- request 1 sent
--- response 1 received
--- request 2 sent
--- response 2 received
CPU times: user 47.7 ms, sys: 7.25 ms, total: 54.9 ms
Wall time: 2.43 s


### Threads

A thread is an execution context, which is all the information a CPU needs to execute a stream of instructions.

Switching context every sys.getswitchinterval() (5ms default)

In [42]:
import sys


print(sys.getswitchinterval())

0.005


In [43]:
%%time

from functools import partial
from threading import Thread


thread_1 = Thread(target=partial(job, n=1))
thread_2 = Thread(target=partial(job, n=2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

--- request 1 sent
--- request 2 sent
--- response 1 received
--- response 2 received
CPU times: user 37.3 ms, sys: 14.2 ms, total: 51.5 ms
Wall time: 1.09 s


### Async

In [44]:
async def ajob(client, n):
    print(f"--- request {n} sent")
    await client.get(f"https://example.com/{n}")
    print(f"--- response {n} received")

In [47]:
import asyncio
import time


async def amain():
    async with httpx.AsyncClient() as client:
        await asyncio.gather(
            ajob(client=client, n=1),
            ajob(client=client, n=2)
        )


start = time.time()
await amain()
print(time.time() - start)

--- request 1 sent
--- request 2 sent
--- response 1 received
--- response 2 received
0.8680598735809326


## Why do we need async?

Cons of threads:
- threads are heavier
- switching is not under our control (*)

Cons of async:
- special syntax (steeper learning curve)
- need ioloop to execute
- no blocking code allowed (*)

## Timeline

